In [ ]:
from google.colab import  drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from pandas import read_csv
import math
from sklearn.preprocessing import MinMaxScaler

In [ ]:
name=["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w"]
data=read_csv("/content/drive/My Drive/sc_lab/Heart_Dataset.csv",names=name)
data=pd.DataFrame(data)
scaler = MinMaxScaler()
scaler.fit(data)
data

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w
0,1,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0
1,1,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1
2,1,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1
4,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
76,0,1,0,0,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,1,1,0,0
77,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
78,0,0,0,1,1,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,1,1


In [ ]:
d1=data[["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v"]]
d2=data["w"]
d1=np.array(d1)
d2=np.array(d2)
data=np.array(data)

In [ ]:
n=len(d1[0])
n

22

In [ ]:
def fun(input_data,target,weight_input,weight_hidden,bias_hidden,bias_output,L,no_hidden,no_output):

  for d in range(len(input_data)):
    net_input_hidden=[0.0 for p in range(no_hidden)]
    output_hidden=[0.0 for p in range(no_hidden)]

    for j in range(no_hidden):
      for i in range(n):
        net_input_hidden[j]+=input_data[d][i]*weight_input[j][i]
      net_input_hidden[j]+=bias_hidden[j]
      output_hidden[j]=1/(1+math.exp(-(net_input_hidden[j])))

    net_input_outputlayer=[0 for p in range(no_output)]
    output_outputlayer=[0 for p in range(no_output)]

    for k in range(no_output):
      for j in range(no_hidden):
        net_input_outputlayer[k]+=output_hidden[j]*weight_hidden[k][j]
      net_input_outputlayer[k]+=bias_output[k]
      output_outputlayer[k]=1/(1+math.exp(-(net_input_outputlayer[k])))

    error_output=[0 for p in range(no_output)]

    for j in range(no_output):
      error_output[j]=output_outputlayer[j]*(1-output_outputlayer[j])*((target[j]-output_outputlayer[j]))
    
    error_hidden=[0 for p in range(no_hidden)]

    for j in range(no_hidden):
      temp=0
      for k in range(no_output):
        temp+=weight_hidden[k][j]*error_output[k]

      error_hidden[j]=output_hidden[j]*(1-output_hidden[j])*temp

    for j in range(no_hidden):
      for k in range(no_output):
        weight_hidden[k][j]=weight_hidden[k][j]+L*error_output[k]*output_hidden[j]

    for j in range(n):
      for k in range(no_hidden):
        weight_input[k][j]=weight_input[k][j]+L*error_hidden[k]*input_data[d][j]

    
    for j in range(no_hidden):
      bias_hidden[j]=bias_hidden[j]+L*error_hidden[j]
    
    for j in range(no_output):
      bias_output[j]=bias_output[j]+L*error_output[j]

  return weight_input,weight_hidden,bias_hidden,bias_output



In [ ]:
def classification_report(cm):
    accuracy = 0
    total = np.sum(cm)
    diagonal = np.sum(np.diag(cm))
    accuracy=(diagonal/total)*100
    print("Accuracy : => ",(diagonal/total)*100)
    return accuracy

In [ ]:
no_hidden=5
no_output=1
block=int(len(data)/10)
M=[[ 0 for i in range(2)] for i in range(2)]

for i in range(10):

  weight_input=[[1/(n*5) for i in range(n)] for j in range(no_hidden)]
  weight_hidden=[[1/5 for i in range(no_hidden)] for j in range(no_output)]
  bias_hidden=[1/6 for i in range(no_hidden)]
  bias_output=[1/6 for i in range(no_output)]
  L=0.2
  temp_x_data=[]
  temp_y_data=[]
  
  x_test=[]
  y_test=[]
  

  point=[]
  for j in range(block):
    point.append(block*i+j)

  
  for j in range(len(data)):
    if(j in point):
      x_test.append(d1[j])
      y_test.append(d2[j])
    else:
      temp_x_data.append(d1[j])
      temp_y_data.append(d2[j])


  
  matrix=[[0 for i in range(2)]for i in range(2)]
  
  for j in range(100):
    w1,w2,b1,b2=fun(temp_x_data,temp_y_data,weight_input,weight_hidden,bias_hidden,bias_output,L,no_hidden,no_output)
    weight_input,weight_hidden,bias_hidden,bias_output=w1,w2,b1,b2
  
  for x in range(len(x_test)):
    net_input_hidden=[0.0 for p in range(no_hidden)]
    output_hidden=[0.0 for p in range(no_hidden)]

    for k in range(no_hidden):
      for j in range(len(x_test[x])):
        net_input_hidden[k]+=x_test[x][j]*weight_input[k][j]
      net_input_hidden[k]+=bias_hidden[k]
      output_hidden[k]=1/(1+math.exp(-(net_input_hidden[k])))

    net_input_outputlayer=[0 for p in range(no_output)]
    output_outputlayer=[0 for p in range(no_output)]

    for k in range(no_output):
      for j in range(no_hidden):
        net_input_outputlayer[k]+=output_hidden[j]*weight_hidden[k][j]
      net_input_outputlayer[k]+=bias_output[k]
      output_outputlayer[k]=1/(1+math.exp(-(net_input_outputlayer[k])))
    c=0
    if(output_outputlayer[0]>0.5):
      c=1
    
    matrix[y_test[x]][c]+=1

  for p in range(2):
    for q in range(2):
      M[p][q]+=matrix[p][q]
  print("Fold ",i+1)
  print(matrix)
  accu=classification_report(matrix)

print("Final confusion matrix")
print(M)
print()
print("Average accuracy=>",classification_report(M))

Fold  1
[[0, 3], [0, 5]]
Accuracy : =>  62.5
Fold  2
[[2, 0], [6, 0]]
Accuracy : =>  25.0
Fold  3
[[6, 0], [2, 0]]
Accuracy : =>  75.0
Fold  4
[[6, 0], [2, 0]]
Accuracy : =>  75.0
Fold  5
[[4, 0], [4, 0]]
Accuracy : =>  50.0
Fold  6
[[6, 0], [2, 0]]
Accuracy : =>  75.0
Fold  7
[[6, 0], [2, 0]]
Accuracy : =>  75.0
Fold  8
[[7, 0], [1, 0]]
Accuracy : =>  87.5
Fold  9
[[7, 0], [1, 0]]
Accuracy : =>  87.5
Fold  10
[[7, 0], [1, 0]]
Accuracy : =>  87.5
Final confusion matrix
[[51, 3], [21, 5]]

Accuracy : =>  70.0
Average accuracy=> 70.0
